## **Installing Libraries**

In [1]:
!pip install python_speech_features

In [2]:
#  os and argparse is done to read files from local folders
import os
import argparse

import numpy as np
from scipy.io import wavfile
import scipy.signal
from python_speech_features import mfcc
import matplotlib.pyplot as plt
import warnings
import random

# tesorflow is for the Deep learning model
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense ,LSTM , TimeDistributed
from tensorflow.keras.layers import Dropout
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import IPython
import librosa
from scipy.signal import butter, lfilter
%matplotlib inline

In [3]:
from google.colab import drive  # the sounds are stored in google drive 
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# folder where files are stored
file = "dataset/training-a/wav/"

**about the dataset**

In [4]:
#############################################################  MFCC EXTRACTION   ##########################################################
# Variable assignment 
tt = 0
time_steps = 450
data_directory=os.getcwd()
nfft = 1203 # Number of FFTs

digit_directory = data_directory


# To normalize the signal
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm
    
def process_directory():
    mfcc_features = list()
    for filename in [x for x in os.listdir(data_directory) if x.endswith('.wav')]:
        # Read the .wav input file
        filepath = os.path.join(digit_directory, filename)
        sampling_freq, audio = wavfile.read(filepath)
        label="n.wav"


# open the .hea file of the same filename to get the label as normal /abnormal.
        st= data_directory +"/"+filename.split(".")[0]+".hea"
        with open(st,'r') as f:
            for line in f:
                for word in line.split():
                    if(word=="Abnormal"):
                        label="a.wav"


# now we have the label stored in 'label' and the audio as 'audio' with sampling freq. as 'sampling_freq'.
        #audio1 = audio[dic[filename.split(".")[0]]:]
        audio1 = SVDnoise(audio/32768)
        #audio1 = audio
        temp = mfcc(audio1, sampling_freq, nfft=nfft)
        temp = temp[tt:tt+time_steps,:]
        mfcc_features.append({"label": label, "mfcc": temp })


        # mfcc features of this audio has been appended to the list 
    return mfcc_features


###########################   CREATING MFCC FEATURES   ############################
processed_files = list()
mfcc_features = process_directory()
random.shuffle(mfcc_features)

############   TRAINING DATA   ###########
size = (8*len(mfcc_features))/10
train_features = mfcc_features[0:int(size)]
test_list = mfcc_features[int(size+1):]
train_size = 0
for feature in train_features:
    train_size += 1
    processed_files.append({'label': feature["label"], 'feature': feature["mfcc"] })
# Train rnn for each MFCC and add to training set
x = np.zeros((train_size, time_steps ,13))
y_train = np.zeros((train_size))
i = 0
for processed_file in processed_files:
#       print(processed_file['label'])
#       print(processed_file['feature'].shape)
    x[i,:,:] = processed_file['feature']
    s = processed_file['label']
    if(s[0]=='a'):
        y_train[i]=1
    else:
        y_train[i]=0
    i += 1
normalize(x)

############   TESTING DATA   #############
test_files = list()
test_features = test_list
test_size = 0
for feature in test_features:
    test_size += 1
    test_files.append({'label': feature["label"], 'feature': feature["mfcc"] })
y_test = np.zeros((test_size))
x_test = np.zeros((test_size, time_steps ,13))
i = 0
for test_file in test_files:
    x_test[i,:,:] = test_file['feature']
    s = test_file['label']
#             print(s)
    if(s[0]=='a'):
        y_test[i]=1
    else:
        y_test[i]=0
    i += 1
normalize(x_test)

print('MFCC EXTRACTION DONE !')

MFCC EXTRACTION DONE !


## RNN MODEL
### Model Structure:
- visible layer or input layer : size of input = 13 (mfcc matrix has column size 13)
- hidden layer 1: LSTM layer
- hidden layer 2: LSTM layer
- 1 dense layer having activation function = "relu" (rectified linear)
- output layer : classification  

![download.png](attachment:download.png)  
<font size="2">*In the figure, the dense layer is missing, but this is to give a brief idea of how the data is flowing in the model structure.*</font>  
### Building the Model:
- loss function: to compute the loss (currently "mean squared error")
- optimizer function: adam
- metrics: accuracy
- **model.fit:** this function tries for the best possible fit of the model to the training data.
<br>
<font size="2"> <font color="brown"> The later part of the code was to try the model for different values of Dopout(lmabda) to calculate accuracy.</font>


In [5]:
dropouts = np.array([0.15])
accuracy = np.zeros(len(dropouts),dtype=float)

In [6]:
#########################  VARIABLES  ######################
cell_no = 13
Epoch_size = 100
Lambda = 0.029  # dropout variable for regularization
# No. of LSTM layers =2
# Cost func. = cosh

####################   MODEL STRUCTURE  ####################
visible=Input(shape=(None,13))
hidden11 = LSTM(cell_no,return_sequences=True)(visible)
hidden1=Dropout(Lambda)(hidden11)

hidden22 = LSTM(cell_no)(hidden1)
hidden2=Dropout(Lambda)(hidden22)

hidden33 = Dense(10, activation='relu')(hidden2)
hidden3 = Dropout(Lambda)(hidden33)
# hidden4 = TimeDistributed(Dense(1))

output = Dense(1, activation='sigmoid')(hidden3)
# output=Dropout(Lambda)(output1)

model = Model(inputs=visible, outputs=output)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
##################   MODEL END ########################
count = 0
for dr in dropouts:
    total=0
    for ii in range(1):
        Lambda = dr
        print(dr)

        model.fit(x, y_train, epochs=Epoch_size, batch_size=10,verbose=1)
        predict=model.predict(x_test)
        y_pred=(predict>0.1)
        acc=model.evaluate(x_test,y_test,verbose=0)
        # print(predict)

        total += acc[1]
        print(acc)
    total /=1
    accuracy[count] = total
    count += 1

0.15
Epoch 1/100


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:

#Print f1, precision, and recall scores
print(precision_score(y_test, y_pred , average="macro"))
print(recall_score(y_test, y_pred , average="macro"))
print(f1_score(y_test, y_pred , average="macro"))
confusion = confusion_matrix(y_test,y_pred,labels=[0,1])
print(confusion)

0.4661971830985916
0.48095238095238096
0.4614626939177543
[[ 2 19]
 [ 8 52]]


In [7]:
accuracy

array([0.])